# Bradford 0-19 Children and Young Peoples' Outcomes Framework: Data Cleaning

# Load data

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from sqlalchemy import create_engine
from IPython.display import display
from rich import print
from pandas.api.types import is_datetime64_any_dtype

from utils import (
    show_person_coverage, 
    report_changes, 
    count_records_and_person, 
    report_value_counts, 
    display_asq_dist,
)

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#| echo: false

# Note: The connection string is specific to the Connected Bradford VDE.
# Replace the placeholder below with the internal server URL.
conn_str = "DATABASE_URL_PLACEHOLDER"

# Create SQLAlchemy engine
engine = create_engine(conn_str)

tbl_name = "person_linked_2016plus"

df = pd.read_sql(f"SELECT * FROM [dbo].[{tbl_name}]", engine)

# Data cleaning

- Removing completely identical records.

- Children with Conflicting ASQ Values (ignoring ASQ date): 
  
  ➡️ **Data handling approach**: 
    - Where one person has more than two entries for the ASQ, and these were completed on the same date, do not use either score in the analyses (but retain other valid ASQ records for the same person).
    - Where one person has more than two entries for the ASQ, and these were completed on different dates, use the most recent score.
    
      *Note: This is the case regardless of whether the score is a 0 or a non-0.*

- Children with multiple 2y home visit records:
  
  ➡️ **Data handling approach**: Keep the latest one completed before the child reached 30 months of age (≈ 914 days).

- If records are neither ASQ-3 nor ASQ:SE, or do not have domain context.
  
  ➡️ **Data handling approach**: Remove all of them.

## Remove duplicates

In [ ]:
init_num_records = df.shape[0]
init_num_person = df['person_id'].nunique()

total_records_before, num_unique_person_before = count_records_and_person(df)

df.drop_duplicates(inplace=True)

total_records_after, num_unique_person_after = count_records_and_person(df)

print(f"Removed {total_records_before - total_records_after:,} fully identical duplicate records, keeping one copy from each set.")

report_changes(
	total_records_before, total_records_after, num_unique_person_before, num_unique_person_after, 
	caption="Changes after removing duplicates", 
	mode='style'
)

## Keep the latest 2y home visit completed before the child reached 30 months of age (≈ 914 days).

In [ ]:
assert is_datetime64_any_dtype(df["HV_DateEvent"]), \
    f"Type Error: Column 'HV_DateEvent' is {df['HV_DateEvent'].dtype}, but expected a datetime type."

assert is_datetime64_any_dtype(df["age_2_5"]), \
    f"Type Error: Column 'age_2_5' is {df['age_2_5'].dtype}, but expected a datetime type."

# Count how many Home Visit (HV) records each child has 
hv_counts = df.groupby("person_id")["HV_DateEvent"].nunique(dropna=True)

# Identify children who have more than one HV record
multi_hv_ids = hv_counts[hv_counts > 1].index
no_hv_before = hv_counts.eq(0).sum()

print(f"{len(multi_hv_ids):,} children have multiple 2-year Home Visit records.")
print(f"{hv_counts.eq(1).sum():,} children have exactly one 2-year Home Visit record.")
print(f"{no_hv_before:,} children have no recorded 2-year Home Visit.")

total_records_before, num_unique_person_before = count_records_and_person(df)

# Keep all records where:
#   - The Home Visit date is missing (no HV done), OR
#   - The Home Visit was completed before the child reached 30 months of age
df_valid_hv = df[
    (df["HV_DateEvent"].isna()) |
    (df["HV_DateEvent"] <= df["age_2_5"])
].copy()

n_no_hv = df_valid_hv.loc[df_valid_hv["HV_DateEvent"].isna(), "person_id"].nunique()

n_valid_hv = df_valid_hv.loc[
    df_valid_hv["HV_DateEvent"].notna(),
    "person_id"
].nunique()

n_total = df_valid_hv.person_id.nunique()

print(f"{n_total:,} children had no HV record OR had an HV on/before the 2.5-year cutoff date.")
print(f"  • {n_no_hv:,} children had no HV record.")
print(f"  • {n_valid_hv:,} children had a valid ≤2y HV record.")

# For children with multiple Home Visit records within the 30-month window,
#     keep only the latest (most recent) one per person
df_latest_hv = (
    df_valid_hv
    .sort_values(["person_id", "HV_DateEvent"], ascending=[True, False], na_position='last')
    .drop_duplicates(subset="person_id", keep="first")
    [["person_id", "HV_CTV3Code", "HV_CTV3Text", "HV_DateEvent"]]
)

df = df.drop(columns=['HV_CTV3Code', 'HV_CTV3Text', 'HV_DateEvent'], errors='ignore').drop_duplicates()

df = pd.merge(
    df,
    df_latest_hv,
    how='left',
    on='person_id'
)

total_records_after, num_unique_person_after = count_records_and_person(df)

report_changes(
	total_records_before, total_records_after, num_unique_person_before, num_unique_person_after, 
	caption="Changes after filtering Home Visit records within 30 months (latest per child)", 
	mode='style'
)

hv_within = df[df["HV_DateEvent"].notna()]
hv_counts_after = df.groupby("person_id")["HV_DateEvent"].nunique(dropna=True)
no_hv_after = hv_counts_after.eq(0).sum()

assert no_hv_before <= no_hv_after, "Some children with no HV records were lost!"
# print(f"Verified: all {no_hv_after:,} children with no HV records were successfully retained.")

assert df.groupby("person_id")["HV_DateEvent"].nunique(dropna=True).max() <= 1, \
    "Some children still have multiple Home Visit records after filtering."

assert df.groupby('person_id')[['HV_CTV3Code', 'HV_DateEvent']].nunique(dropna=True).le(1).all(axis=1).all(), \
    "Some persons have multiple HV entries."

print(f"{hv_within['person_id'].nunique():,} children have valid Home Visit records within 30 months (≤ 914 days).")
print(f"{df['person_id'].nunique():,} total children retained, including {no_hv_after:,} without any valid 2y Home Visit record.")

In [ ]:
df_stud = show_person_coverage(
	df, 
	cols=["HV_DateEvent", "ASQ_value", "birth_datetime", "ethnicity_raw", "gender_raw", 'FSP_GLD', 'FSP_LSOA11', 'death_datetime'], 
	max_height=300, 
	return_df=True,
	show_html=False
)

df_stud.style.set_caption("Coverage metrics (children with values vs. missing) across key variables - after cleaning HV records")

## Remove records with multiple ASQ values completed on the save date

In [ ]:
group_keys = ['person_id', 'ASQ_CTV3Code', 'ASQ_DateEvent']

# Identify records where a person has conflicting non-0 ASQ scores 
# (same ASQ domain and date, but more than one unique ASQ_value)
multi_valued = df.groupby(group_keys).filter(
	lambda g: g['ASQ_value'].dropna().nunique() > 1
).reset_index(drop=False)

print(f"There are {multi_valued.shape[0]:,} records with multiple ASQ values "
      f"on the same date, involving {multi_valued['person_id'].nunique():,} unique persons. "
      f"All will be removed.")

if multi_valued.shape[0] > 0:
	total_records_before, num_unique_person_before = count_records_and_person(df)

	# Pick one example case
	example_row = multi_valued.iloc[0]   # take the first conflict case
	example_id = example_row['person_id']
	example_code = example_row['ASQ_CTV3Code']
	example_date = example_row['ASQ_DateEvent']
	
	example_before = df.loc[
	    (df['person_id'] == example_id) &
	    (df['ASQ_CTV3Code'] == example_code) &
	    (df['ASQ_DateEvent'] == example_date),
	    group_keys + ['ASQ_value']
	]

	df = df.drop(index=multi_valued['index'])

	total_records_after, num_unique_person_after = count_records_and_person(df)

	report_changes(
		total_records_before, total_records_after, num_unique_person_before, num_unique_person_after, 
		caption="Changes after removing multi-valued ASQ entries on same date", 
		mode='style'
	)

 	# Validation
	assert df.groupby(group_keys)['ASQ_value'].nunique().max() <= 1, \
    "Error: Still found conflicting values on the same date!"

	print("Validation passed: No person has multiple ASQ values on the same date after removal.")
   
   	# Show one person's records before and after removal
	print("\n--- Example: one person's records BEFORE removal ---")
	display(example_before)
	
	print("\n--- Example: one person's records AFTER removal ---")
	example_after = df.loc[
		(df['person_id'] == example_id) &
		(df['ASQ_CTV3Code'] == example_code) &
		(df['ASQ_DateEvent'] == example_date),
		group_keys + ['ASQ_value']
	]
	# display(example_after)

	assert len(example_after) == 0, f"Example records still exist in the dataset!\n {example_after}"
 
	print("Validation passed: This person's records with multiple ASQ values on the same date have been removed.")
	
else:
	print("No multi-valued ASQ entries on the same date found.")

In [ ]:
if total_records_before != total_records_after:

	df_stud = show_person_coverage(
		df, 
		cols=["HV_DateEvent", "ASQ_value", "birth_datetime", "ethnicity_raw", "gender_raw", 'FSP_GLD', 'FSP_LSOA11'], 
		max_height=300, 
		return_df=True,
		show_html=False
	)

	df_stud.style.set_caption("Coverage metrics (children with values vs. missing) across key variables - after removing duplicates and multi-valued ASQ entries on same date")

## Retain latest ASQ value for records with mulitple ASQ completed before the child reached 30 months of age (≈ 914 days).

In [ ]:
df_ = df.loc[
        df['ASQ_CTV3Text'].str.lower().str.startswith(("asq-3", "asq third"), na=False)
        & df['ASQ_CTV3Text'].str.contains(r'\b(24|27|30)\b', na=False)
    ]

print('df_ ', df_.shape)

df.drop(columns=['ASQ_CTV3Code', 'ASQ_CTV3Text', 'ASQ_DateEvent', 'ASQ_value'], inplace=True, errors='ignore')
df.drop_duplicates(inplace=True)

df = pd.merge(
    df,
    df_[['person_id', 'ASQ_CTV3Code', 'ASQ_CTV3Text', 'ASQ_DateEvent', 'ASQ_value']].drop_duplicates(),
    how='left',
    on='person_id'
)

del df_

df = display_asq_dist(df)

group_keys = ['person_id', 'ASQ_CTV3Code']

# Identify records where a person has conflicting non-0 ASQ scores 
# (same ASQ domain but more than one unique ASQ_value across dates)
multi_valued = df.groupby(group_keys).filter(
    lambda g: 
        (g['ASQ_value'].dropna().nunique() > 1) |
        (g['ASQ_DateEvent'].dropna().nunique() > 1)
)

print(f"There are {multi_valued.shape[0]:,} records with multiple ASQ values across dates, "
      f"involving {multi_valued['person_id'].nunique():,} unique persons. "
      f"Only the most recent scores will be kept.")

# When a person has multiple ASQ scores for the same domain,
# keep only the one from the latest assessment date within 30 months
multi_latest = (
    multi_valued[multi_valued['ASQ_DateEvent'] <= multi_valued['age_2_5']]
    .sort_values('ASQ_DateEvent')
    .groupby(group_keys)
    .tail(1)
    .reset_index(drop=True)
)

example_row = multi_valued.iloc[0]
example_id = example_row['person_id']
example_code = example_row['ASQ_CTV3Code']

example_before = df.loc[
    (df['person_id'] == example_id) &
    (df['ASQ_CTV3Code'] == example_code),
    ['person_id', 'ASQ_CTV3Code', 'ASQ_DateEvent', 'ASQ_CTV3Text', 'ASQ_value']
]

total_records_before, num_unique_person_before = count_records_and_person(df)

# Drop all conflicting entries, then add back only the latest
df = df.drop(index=multi_valued.index)
df = pd.concat([df, multi_latest], ignore_index=True)

total_records_after, num_unique_person_after = count_records_and_person(df)

# Validation
assert df.groupby(group_keys)[['ASQ_value', 'ASQ_DateEvent']].nunique(dropna=True).le(1).all(axis=1).all(), \
    "Some persons still have more than one unique ASQ entry per domain after keeping the latest."

print("Validation passed: Only the most recent ASQ entry per domain has been kept after cleaning.")

# Show example person's records before and after removal
print("\n--- Example BEFORE ---")
display(example_before)

print("\n--- Example AFTER (only latest completed before the child reached 30 months of age) ---")
example_after = df.loc[
    (df['person_id'] == example_id) &
    (df['ASQ_CTV3Code'] == example_code),
    ['person_id', 'ASQ_CTV3Code', 'ASQ_DateEvent', 'ASQ_CTV3Text', 'ASQ_value']
]

display(example_after)

report_changes(
	total_records_before, total_records_after, num_unique_person_before, num_unique_person_after, 
	caption="Changes after removing multi-valued ASQ entries on same date", 
	mode='style'
)

In [ ]:
if total_records_before != total_records_after:

	df_stud = show_person_coverage(
		df, 
		cols=["HV_DateEvent", "ASQ_value", "birth_datetime", "ethnicity_raw", "gender_raw", 'FSP_GLD', 'FSP_LSOA11'], 
		max_height=300, 
		return_df=True,
		show_html=False
	)

	df_stud.style.set_caption("Coverage metrics (children with values vs. missing) across key variables - after removing duplicates and multi-valued ASQ entries on different dates")

## Remove records are not ASQ-3, or do not have domain context.

We identified some records in the ASQ dataset that could not be classified as either **ASQ-3** or **ASQ:SE**. The classification was based on the `ASQ_CTV3Text` field:

- **ASQ-3 records** were expected to have `ASQ_CTV3Text` (converted to lowercase) starting with *"asq-3"* or *"asq third"*.  

Since such records cannot be reliably linked to the intended ASQ instruments, they were removed from the dataset.

To ensure that no relevant ASQ records are mistakenly excluded, we first examine the raw distribution of `ASQ_CTV3Text` values before any filtering.

In [ ]:
df = display_asq_dist(df, text_col='ASQ_CTV3Text', code_col='ASQ_CTV3Code')

For ASQ, the `ASQ_CTV3Text` values follow certain patterns. For ASQ-3, most of them appear as either `ASQ-3 {age} month questionnaire - {subdomain}`, or `ASQ Third Edition - {subdomain}`.  

Therefore, it is safe to classify records as ASQ-3 based on these two patterns.

I also double-checked the `CTV3Code` values for ASQ-3 without age and for other ASQ records. The code `XaXgI` and `Y2c3c` was not used in Kate's work. For ASQ-3 (no month), Kate also treated these as ASQ records without age, and it seems they were not used either. We may consider removing them as well.

In [ ]:
# only consider non-null values, then check if their ASQ_CTV3Text start with 'asq-3' or 'asq third'
mask_asq3 = df['ASQ_CTV3Text'].notna() & df['ASQ_CTV3Text'].str.lower().str.startswith(('asq-3', 'asq third'))

# only consider non-null values, then check if their ASQ_CTV3Text start with 'asq-se' or 'asq:se'
mask_asqse = df['ASQ_CTV3Text'].notna() & df['ASQ_CTV3Text'].str.lower().str.startswith(('asq-se', 'asq:se'))

df_asq3 = df[mask_asq3].copy()
df_asqse = df[mask_asqse].copy()

# find non-null values that are NOT ASQ-3 or ASQ:SE
# null values are remain untouched
asq_index = df_asq3.index.union(df_asqse.index)
df_other = df[df['ASQ_CTV3Text'].notna() & ~df.index.isin(asq_index)]

if len(df_other) > 0:
	print(
		f"{len(df_other):,} records are neither ASQ-3 nor ASQ:SE, or do not have domain context. Removing them from the dataset."
  		"\nBelow is the distribution of their ASQ_CTV3Text values:"
	)
	
	report_value_counts(
		df_other[['ASQ_CTV3Code', 'ASQ_CTV3Text']], ['ASQ_CTV3Code', 'ASQ_CTV3Text'], 
		mode='style', 
  		caption="Distribution of ASQ_CTV3Code and ASQ_CTV3Text for non-ASQ-3/ASQ:SE records"
	)

	total_records_before, num_unique_person_before = count_records_and_person(df)
	
	df = df.drop(index=df_other.index)

	total_records_after, num_unique_person_after = count_records_and_person(df)
	
	report_changes(
		total_records_before, total_records_after, num_unique_person_before, num_unique_person_after, 
		caption="Changes after removing non-ASQ-3/ASQ:SE records", 
		mode='style'
	)
	
else:
    print("No records are neither ASQ-3 nor ASQ:SE, or do not have domain context.")

In [ ]:
if len(df_other) > 0:
    
    df_stud = show_person_coverage(
		df, 
		cols=["HV_DateEvent", "ASQ_value", "birth_datetime", "ethnicity_raw", "gender_raw", 'FSP_GLD', 'FSP_LSOA11'], 
		max_height=300, 
		return_df=True,
		show_html=False
	)

    df_stud.style.set_caption("Coverage metrics (children with values vs. missing) across key variables - after removing non-ASQ-3/ASQ:SE records")

## Retaining the EYFSP records from the academic year in which the child turns 5

According to the [Early Years Foundation Stage Profile Handbook](https://www.gov.uk/government/publications/early-years-foundation-stage-profile-handbook/early-years-foundation-stage-profile-handbook?utm_source=chatgpt.com):

> *"The EYFS framework requires the EYFS profile assessment to be carried out in the final term of the academic year in which a child reaches age 5, or, in exceptional cases, the final term of the year before the child moves into year 1 if the child remains in EYFS provision beyond age 5."*

So **we retain only the EYFSP record from the academic year in which the child turns 5**, as this represents the valid and legally mandated assessment. Any additional EYFSP entries from later academic years (e.g., age 5-6) are treated as duplicates or administrative errors and excluded from analysis.

In [ ]:
# Identify children with conflicting EYFSP entries
problem_ids = (
    df.groupby('person_id')[['FSP_GLD','FSP_AGE']]
      .nunique(dropna=True)
      .gt(1)                          # >1 unique value
      .any(axis=1)                    # either GLD or AGE has >1
      .pipe(lambda s: s[s].index)     # extract person_ids cleanly
)

print(f"{len(problem_ids):,} children had multiple EYFSP entries.")

if len(problem_ids) > 0:

  # Preview conflicting records (before cleaning)
  preview_cols = ['person_id', 'birth_datetime', 'FSP_ACADYR', 'FSP_GLD', 'FSP_AGE', 'FSP_AGE_START']

  display(
      df.loc[df['person_id'].isin(problem_ids), preview_cols]
        .drop_duplicates()
        .style.hide(axis='index')
        .set_caption("Children with multiple EYFSP entries (before cleaning)")
  )

  # Drop EYFSP records where the child was aged 5 at EYFSP start
  df = df.drop(
      df.loc[df['person_id'].isin(problem_ids) & (df['FSP_AGE_START'] == 5)].index
  )

  # Preview again (after cleaning)
  display(
      df.loc[df['person_id'].isin(problem_ids), preview_cols]
        .drop_duplicates()
        .style.hide(axis='index')
        .set_caption("Children with multiple EYFSP entries (after cleaning)")
  )
  
  df_stud = show_person_coverage(
  	df, 
  	cols=["HV_DateEvent", "ASQ_value", "birth_datetime", "ethnicity_raw", "gender_raw", 'FSP_GLD', 'FSP_LSOA11'], 
  	max_height=300, 
  	return_df=True,
  	show_html=False
  )

  df_stud.style.set_caption("Coverage metrics (children with values vs. missing) across key variables - after removing non-ASQ-3/ASQ:SE records")

## Standardise categorical variables: gender recoding

We standardised the `gender` variable to ensure consistency across different coding schemes present in the dataset. The raw values included multiple formats such as `"F"`, `"M"`, `"1"`, `"2"`, `"Male"`, `"Female"`, `"N"`, `"I"`, `"NA"`, and missing values. To harmonise these, we applied the following mapping:

- `"M"`, `"Male"`, `"1"` → **Male**  
- `"F"`, `"Female"`, `"2"` → **Female**  
- `"N"`, `"I"`, `"NA"`, and missing values (including `None`, blank entries, or empty strings)→ **Unknown/Other**

In [ ]:
gender_map = {
    'm': 'Male',
    'male': 'Male',
    '1': 'Male',
    'f': 'Female',
    'female': 'Female',
    '2': 'Female',
    'n': 'Unknown/Other',
    'i': 'Unknown/Other',
    'na': 'Unknown/Other',
    None: 'Unknown/Other'
}

df['gender'] = df['gender_raw'].str.lower().map(gender_map).fillna('Unknown/Other')

report_value_counts(
    df[['person_id', 'gender_raw', 'gender']], 
    ['gender_raw', 'gender'], 
    mode='style', 
    caption="Raw and cleaned values of 'gender'"
)

report_value_counts(
    df[['person_id', 'gender']], 
    ['gender'], 
    mode='style', 
    caption="leaned values of 'gender'"
)

In [ ]:
df_stud = show_person_coverage(
	df, 
	cols=["HV_DateEvent", "ASQ_value", "birth_datetime", "ethnicity_raw", "gender", 'FSP_GLD', 'FSP_LSOA11'], 
	max_height=300, 
	return_df=True,
	show_html=False
)

df_stud.style.set_caption("Coverage metrics (children with values vs. missing) across key variables - after standardising gender values")

## Within-individual imputation of demographic variables

For key demographic fields (e.g., gender, ethnicity, postcode), if a person has missing values in some rows, we impute the missing values using known values from other rows with the same person ID, assuming demographic stability over time. We also check how many children show inconsistencies in these fields across different records.

In [ ]:
demo_cols = ["gender", "ethnicity_raw", "birth_datetime"]

df_out = df.copy() # copy to avoid modifying original df

for col in demo_cols:
    print(f"\n=== Checking column: {col} ===")

    nunique_nonnull = (
        df.groupby("person_id")[col]
          .apply(lambda x: x.dropna().nunique())
    ) # e.g., ["Male", "Male", NaN] → 1

    nunique_inclu_null = (
        df.groupby("person_id")[col]
          .apply(lambda x: x.nunique())
    ) # e.g., ["Male", "Male", NaN] → 2
    

    conflict_ids = nunique_nonnull[nunique_nonnull > 1].index
    if len(conflict_ids) > 0:
        print(f"{len(conflict_ids)} children have conflicting non-null values in {col}.")
        display(df[df["person_id"].isin(conflict_ids)][["person_id", col]].head(20))
    else:
        print(f"No conflicts in {col}.")
        print(f"No children appears with more than one {col.replace('_raw', '')}.")

        mixed_valid_null_ids = nunique_inclu_null[nunique_inclu_null > nunique_nonnull].index
        if len(mixed_valid_null_ids) > 0:
            # Children with both NaN and valid values (but no conflicts)
            print(f"{len(mixed_valid_null_ids)} children have valid + null entries in {col}.")

            print("Impute missing values where possible...")
            one_value_ids = nunique_nonnull[nunique_nonnull == 1].index
            df_out.loc[df_out["person_id"].isin(one_value_ids), col] = (
                df_out.groupby("person_id")[col].transform(lambda x: x.ffill().bfill())
            )
            

    all_missing_ids = nunique_nonnull[nunique_nonnull == 0].index
    print(f"{len(all_missing_ids)} children have all values missing in {col}.")
    
    if len(all_missing_ids) > 0:
        fill_value = pd.NaT if "datetime" in str(df_out[col].dtype) else "Unknown/Other"
        df_out.loc[df_out["person_id"].isin(all_missing_ids), col] = fill_value
        print(f"Imputed '{col}' as {fill_value} for {len(all_missing_ids)} children.")

# Update final df
df = df_out.copy()

## Remove records with empty `birth_datetime`

In [ ]:
total_records_before, num_unique_person_before  = count_records_and_person(df)

df = df.dropna(subset=['birth_datetime'])

total_records_after, num_unique_person_after = count_records_and_person(df)

print(f"Removing {total_records_before - total_records_after:,} records with empty birth_datetime, involving {num_unique_person_before - num_unique_person_after:,} unique persons.")

if total_records_before - total_records_after > 0:
    report_changes(
        total_records_before, total_records_after, num_unique_person_before, num_unique_person_after, 
        caption="Changes after removing records with empty birth_datetime", 
        mode='style'
    )

## Remove children's records with faulty `death_datetime`

Some children died before birth; the cause is unknown, so remove these records.

In [ ]:
mask = df['death_datetime'].notna() & (df['birth_datetime'] > df['death_datetime'])

bad_ids = df.loc[mask, 'person_id'].unique()

if len(bad_ids) > 0:
	total_records_before, num_unique_person_before = count_records_and_person(df)
	
	display(
		df[df['person_id'].isin(bad_ids)][['person_id', 'birth_datetime', 'death_datetime']]
		.style
		.hide(axis='index')
		.set_caption("Children with faulty `death_datetime`")
	)

	df = df[~df['person_id'].isin(bad_ids)]

	total_records_after, num_unique_person_after = count_records_and_person(df)

	report_changes(
		total_records_before,
		total_records_after,
		num_unique_person_before,
		num_unique_person_after,
		caption="Changes after removing records with faulty `death_datetime`",
		mode='style'
	)

	df_stud = show_person_coverage(
		df,
		cols=["HV_DateEvent", "ASQ_value", "birth_datetime", "ethnicity_raw", "gender_raw", 'FSP_GLD', 'FSP_LSOA11'],
		max_height=300,
		return_df=True,
		show_html=False
	)

	df_stud.style.set_caption(
		"Coverage metrics (children with values vs. missing) across key variables - after removing records with faulty `death_datetime`"
	)

# Feature transformation / derivation

## Deriving ethnicity group

The ethnicity values in the source dataset look like this: `White: English or Welsh or Scottish or Northern Irish or British - England and Wales ethnic category 2011 census`.

To make things easier to work with, we split them into two parts:

- The part **before the colon (`:`)** is used as the **main ethnic group**, e.g. `White`.
- The part **after the colon but before the dash (`-`)** is kept as the **detailed subgroup**, e.g. `English or Welsh or Scottish or Northern Irish or British`.
- Anything **after the dash** is removed, as it is just extra description we don't need.
- Fill missing value with `Unknown/Refuse to say: Unknown/Refuse to say`.

So in the end, we get two new columns:
- `ethnicity_group` → e.g. `White`
- `ethnicity_subgroup` → e.g. `English or Welsh or Scottish or Northern Irish or British`

The following groupings will be used which are based on the majority ethnic populations in Bradford:

- White British (English/Welsh/Scottish/Northern Irish or British)
- White Other (Any other white background)
- Asian/ Asian British Pakistani
<!-- - Asian/ Asian British Bangladeshi -->
- Asian/ Asian British Other (Indian / Chinese / Bangladeshi / any other Asian background)
- Black / African / Carribbean / Black British
- Other (including mixed multiple ethnic groups / any other groups)

In [ ]:
df["ethnicity_tmp"] = df["ethnicity_raw"].replace('Unknown/Refuse to say', 'Unknown/Refuse to say: Unknown/Refuse to say').fillna('Unknown/Refuse to say: Unknown/Refuse to say')
df["ethnicity_tmp"] = df["ethnicity_tmp"].str.extract(r"^([^-]+)")[0]
df[["ethnicity_census_group", "ethnicity_census_subgroup"]] = df["ethnicity_tmp"].str.extract(r"^([^:]+):\s*([^-]+)")

df.drop(columns=["ethnicity_tmp"], inplace=True)

df["ethnicity_census_group"] = (
    df["ethnicity_census_group"]
    .str.strip()
)

df["ethnicity_census_subgroup"] = (
    df["ethnicity_census_subgroup"]
    .str.strip()
)

def map_ethnicity(row):
    g, sg = row['ethnicity_census_group'].lower(), row['ethnicity_census_subgroup'].lower()

    if g == 'white':
        if sg == 'english or welsh or scottish or northern irish or british':
            return 'White British'
        # elif sg == 'Any other White background':
        #     return 'White Other'
        else:
            return 'White Other'
    
    elif g == 'asian or asian british':
        if sg == 'pakistani':
            return 'Asian/Asian British Pakistani'
        
        elif sg == 'bangladeshi':
            # return 'Asian/Asian British Bangladeshi'
            return 'Asian/Asian British Other'
        
        elif sg in ['indian', 'chinese', 'any other asian background']:
            return 'Asian/Asian British Other'
    
    elif g == 'black or african or caribbean or black british':
        return 'Black/African/Caribbean/Black British'
    
    elif g in ['mixed multiple ethnic groups', 'other ethnic group', 'unknown/refuse to say']:
        return 'Other'

    return 'ERROR'  # fallback

df['ethnicity_group'] = df.apply(map_ethnicity, axis=1)

report_value_counts(
    df[["person_id", "ethnicity_census_group", "ethnicity_census_subgroup", "ethnicity_group"]], 
    ["ethnicity_census_group", "ethnicity_census_subgroup", "ethnicity_group"], 
    mode='style', 
    caption="Ethnicity group and subgroup extracted from raw values",
    max_height=300, 
)

report_value_counts(
    df[["person_id", "ethnicity_group"]], 
    ["ethnicity_group"], 
    mode='style', 
    caption="Ethnicity group extracted from raw values",
    max_height=300, 
)

## LSOA → IMD 2019 mapping

### LSOA version check  

In [ ]:
df_oa_lookup = pd.read_csv('Lower_Layer_Super_Output_Area_(2001)_to_Lower_Layer_Super_Output_Area_(2011)_to_Local_Authority_District_(2011)_Lookup_in_England_and_Wales.csv')

df['LSOA_clean'] = (
    df['LSOA'].astype(str).str.strip().replace({'': pd.NA, 'nan': pd.NA, 'None': pd.NA})
)

lsoa_data = (
    df['LSOA_clean']
    .dropna()
    .unique()
    .tolist()
)

lsoa_data = set(lsoa_data)
lsoa_2001 = set(df_oa_lookup['LSOA01CD'])
lsoa_2011 = set(df_oa_lookup['LSOA11CD'])

# --- classify LSOA ---
in_2001 = lsoa_data & lsoa_2001
in_2011 = lsoa_data & lsoa_2011
in_both = in_2001 & in_2011
only_2001 = in_2001 - in_2011
only_2011 = in_2011 - in_2001
in_neither = lsoa_data - (lsoa_2001 | lsoa_2011)

total_lsoa = len(lsoa_data)
valid_lsoa_mask = df['LSOA_clean'].notna()
total_children = df.loc[valid_lsoa_mask, 'person_id'].nunique()

# formatting function
def fmt(count, denom):
    return f"{count} ({count/denom:.2%})"

# children count helper
def count_children(lsoa_set):
    return df[df['LSOA_clean'].isin(lsoa_set)]['person_id'].nunique()

# children counts
n_children_only_2001 = count_children(only_2001)
n_children_only_2011 = count_children(only_2011)
n_children_both = count_children(in_both)
n_children_neither = count_children(in_neither)

lsoa_version_check = pd.DataFrame({
    "category": [
        "total_unique_LSOA",
        "only_in_2001",
        "only_in_2011",
        "in_both",
        "in_neither_2001_2011",
    ],
    "lsoa_count_pct": [
        total_lsoa,
        fmt(len(only_2001), total_lsoa),
        fmt(len(only_2011), total_lsoa),
        fmt(len(in_both), total_lsoa),
        fmt(len(in_neither), total_lsoa),
    ],
    "children_count_pct": [
        total_children,
        fmt(n_children_only_2001, total_children),
        fmt(n_children_only_2011, total_children),
        fmt(n_children_both, total_children),
        fmt(n_children_neither, total_children),
    ]
})

display(
    lsoa_version_check.style.hide(axis='index').set_caption('LSOA Version Summary')
)

neither_list = sorted(in_neither)
neither_str = ", ".join(neither_list)

# unique children whose LSOA not in 2001/2011 lookup
n_person_missing = df[df['LSOA_clean'].isin(in_neither)]['person_id'].nunique()

print(
    f"LSOA codes not found in the England & Wales LSOA 2001 or 2011 versions "
    f"({len(neither_list)} distinct codes):", [", ".join(neither_list)]
)

print(
    f"\nNumber of unique children associated with these non-England/Wales LSOA codes: "
    f"{n_person_missing:,}"
)

### Map LSOA to IMD 2019

In [ ]:
oa_mapping = (
    df_oa_lookup.groupby('LSOA01CD')['LSOA11CD']
          .apply(list)
          .to_dict()
)

df_imd19 = pd.read_excel('File_1_-_IMD2019_Index_of_Multiple_Deprivation.xlsx', sheet_name='IMD2019')
df_imd15 = pd.read_excel('File_1_ID_2015_Index_of_Multiple_Deprivation.xlsx', sheet_name='IMD 2015')

df_imd19_map = df_imd19.set_index('LSOA code (2011)')['Index of Multiple Deprivation (IMD) Decile']
df_imd15_map = df_imd15.set_index('LSOA code (2011)')['Index of Multiple Deprivation (IMD) Decile (where 1 is most deprived 10% of LSOAs)']

df['LSOA_final'] = df['LSOA_clean'].apply(lambda x: oa_mapping.get(x, x) if x in only_2001 else x)

def collapse_imd(x, mode="strict"):
    """
    mode = "strict": if multiple distinct IMD values exist, return NaN.
    mode = "keeplist": if multiple distinct IMD values exist, keep the list.
    """

    # If not a list, return the value as is
    if not isinstance(x, list):
        return x

    # Remove None values
    unique_vals = [v for v in x if v is not None]

    # If only one unique IMD value → return that value
    if len(set(unique_vals)) == 1:
        return unique_vals[0]

    # Multiple different IMD values → handle based on mode
    if mode == "strict":
        return np.nan
    elif mode == "keeplist":
        return unique_vals
    else:
        raise ValueError("mode must be either 'strict' or 'keeplist'")

df['IMD19_deciles'] = df['LSOA_final'].apply(
    lambda x: [df_imd19_map.get(i) for i in x] if isinstance(x, list)
              else df_imd19_map.get(x)
).apply(collapse_imd).astype('Int64')

df['IMD15_deciles'] = df['LSOA_final'].apply(
    lambda x: [df_imd15_map.get(i) for i in x] if isinstance(x, list)
              else df_imd15_map.get(x)
).apply(collapse_imd).astype('Int64')

df['IMD19_deciles_raw'] = (
    df['LSOA_final']
    .apply(lambda x: [df_imd19_map.get(i) for i in x] if isinstance(x, list) else df_imd19_map.get(x))
    .apply(lambda x: collapse_imd(x, mode="keeplist"))
)

df['IMD15_deciles_raw'] = (
    df['LSOA_final']
    .apply(lambda x: [df_imd15_map.get(i) for i in x] if isinstance(x, list) else df_imd15_map.get(x))
    .apply(lambda x: collapse_imd(x, mode="keeplist"))
)

cols = ['LSOA', 'LSOA_final', 'IMD19_deciles', 'IMD15_deciles', 'IMD19_deciles_raw', 'IMD15_deciles_raw']

qa = []

for col in cols:
    qa.append({
        "column": col,
        "missing_rate": df[col].isna().mean(),
        "unique_persons_missing": df[df[col].isna()].person_id.nunique()
    })

qa_df = pd.DataFrame(qa)
display(qa_df.style.hide(axis='index').format({"missing_rate": "{:.2%}"}))

In [ ]:
def count_multi_imd(df, imd_col):
    # LSOA → list of IMD values
    mapping = (
        df.groupby('LSOA')[imd_col]
          .apply(lambda x: {v for lst in x if isinstance(lst, list) for v in lst})
    )

    # LSOA with multiple IMD values
    multi_lsoas = [lsoa for lsoa, vals in mapping.items() if len(vals) > 1]

    # Number of affected children
    affected_children = df[df['LSOA'].isin(multi_lsoas)]['person_id'].nunique()

    return len(multi_lsoas), affected_children


n_multi_19, n_children_19 = count_multi_imd(df, 'IMD19_deciles_raw')
n_multi_15, n_children_15 = count_multi_imd(df, 'IMD15_deciles_raw')

print(f"IMD19: {n_multi_19} LSOA with multiple IMD values, affecting {n_children_19} children.")
print(f"IMD15: {n_multi_15} LSOA with multiple IMD values, affecting {n_children_15} children.")

In [ ]:
df_stud = show_person_coverage(
	df, 
	cols=["HV_DateEvent", "ASQ_value", "birth_datetime", "ethnicity_raw", "gender_raw", 'FSP_GLD', 'FSP_LSOA11', 'IMD19_deciles', 'IMD15_deciles'], 
	max_height=300, 
	return_df=True,
	show_html=False
)

df_stud.style.set_caption("Coverage metrics (children with values vs. missing) across key variables - final view")

In [ ]:
df['LSOA_final'] = df['LSOA_final'].apply(
    lambda x: x if isinstance(x, list) else [x]
)

df['IMD19_deciles_raw'] = df['IMD19_deciles_raw'].apply(
    lambda x: x if isinstance(x, list) else [x]
)

df['IMD15_deciles_raw'] = df['IMD15_deciles_raw'].apply(
    lambda x: x if isinstance(x, list) else [x]
)

## Create IMD Quintiles

In [ ]:
df['IMD19_quintile'] = np.ceil(df['IMD19_deciles'] / 2).astype('Int64')

report_value_counts(
    df[["person_id", "IMD19_quintile", "IMD19_deciles"]], 
    ["IMD19_quintile", "IMD19_deciles"], 
    mode='style', 
    caption="IMD19_deciles and IMD19_deciles mapped from LSOA",
    max_height=300, 
)

In [ ]:
# | echo: false

try:
    df.to_sql(
        name=f'{tbl_name}_cleaned',   
        con=engine,                   
        schema='dbo',                
        if_exists='replace',          
        index=False                 
    )
except Exception as e:
    print("Error occurred while writing to SQL Server.")
    print(str(e).split("\n")[0])

engine.dispose()